

## Compilador PL0

Nicolas Moreno

Kevin Oquendo

# Gramatica

programa : definicion_funcion
          | definicion_main

definicion_funcion : FUN IDENT "(" parameter_list ")" BEGIN statement_list END

definicion_main : FUN MAIN "(" ")" BEGIN statement_list END

parameter_list : parameter ("," parameter)*

parameter : IDENT ":" TYPE ";"

statement_list : statement ";" statement_list
                  | statement

statement : while_statement
          | for_statement
          | if_statement
          | begin_statement
          | assignment_statement
          | read_statement
          | write_statement
          | return_statement
          | skip_statement
          | break_statement
          | expression

while_statement : WHILE "(" expression ")" DO statement_list END

for_statement : FOR IDENT IN range_expression DO statement_list END

if_statement : IF "(" expression ")" THEN statement_list ELSE statement_list END

begin_statement : BEGIN statement_list END

assignment_statement : IDENT ASSIGN_OP expression

read_statement : READ

write_statement : WRITE

return_statement : RETURN expression

skip_statement : SKIP

break_statement : BREAK

expression : term (relational_operator term)

expression : term ("+" term
                  | "-" term)*

term : factor ("*" factor
              | "/" factor)*

factor : IDENT
      | INUMBER
      | FNUMBER
      | "(" expression ")"
      | UNARY
      | BINARY
      | STRING
      | LOCATION

range_expression : "(" expression COMA expression ")"

relational_operator : EQ
                    | NE
                    | GT
                    | LT
                    | GE
                    | LE

array_expression : IDENT LBRACKET expression RBRACKET
                  | IDENT LBRACKET expression COMA expreesion RBRACKET

LBRACKET : "["

RBRACKET : "]"

COMA : ","

DOSPUNTOS : ":"

ASSIGN_OP : ":="

# Analizador lexico


In [ ]:
!pip install sly
!pip install multimethod

In [ ]:
import sly
import math

In [ ]:
from sly import Lexer

class Lexer(Lexer):
    tokens = {
        IDENT, INUMBER, FNUMBER, ASSIGN, NEWLINE, STRING,
        WHILE,FOR, DO, IF, THEN, ELSE, LOCATION, PRINT, WRITE,
        READ, RETURN, SKIP, BREAK, BEGIN, END, FUN, MAIN,
        EQ, NE, GT, LT, GE, LE, LBRACKET, RBRACKET, COMA , DOSPUNTOS
    }
    literals = '()+-*/%=^!;'

    ignore = ' \t'

    @_(r'\n+')
    def ignore_newline(self,t):
        self.lineno += t.value.count('\n')

    @_(r'\/\*(\w|\n)*(\*\/)')
    def ignore_comment(self, t):
        self.lineno += t.value.count('\n')


    @_(r'(\+|\-)?[1-9]\d*|[0]{1}\s')  # Números enteros
    def INUMBER(self, t):
        t.value = int(t.value)
        return t

    @_(r'(([1-9]((\d+\.|\.|)\d+|))((e-?|e\+?)[1-9]\d*)?)|(0\.\d+)|([0]{1}\s)')
    def FNUMBER(self, t):
        t.value = float(t.value)
        return t

    @_(r'[a-zA-Z](\w)*')  # Identificadores
    def IDENT(self, t):
        t.value = t.value
        t.type = t.value.upper()
        return t

   # Palabras clave
    WHILE = r'while'
    FOR = r'for'
    FUN = r'fun'
    MAIN = r'main'
    DO = r'do'
    IF = r'if'
    THEN = r'then'
    ELSE = r'else'
    LOCATION = r'Location'
    PRINT = r'Print'
    WRITE = r'Write'
    READ = r'Read'
    RETURN = r'Return'
    SKIP = r'skip'
    BREAK = r'break'
    BEGIN = r'begin'
    END = r'end'

    DOSPUNTOS = r':'

    # Operadores de comparación
    EQ = r'=='
    NE = r'!='
    GT = r'>'
    LT = r'<'
    GE = r'>='
    LE = r'<='

    #asignacion
    ASSIGN = r'='

    # Cadena de caracteres
    @_(r'"(?:[^"])*"')
    def STRING(self, t):
        t.value = t.value[1:-1]  # Elimina las comillas
        return t

    @_(r',')
    def COMA(self, t):
        return t

    @_(r'\[')
    def LBRACKET(self, t):
        return t

    @_(r'\]')
    def RBRACKET(self, t):
        return t

    # Nueva línea
    @_(r'\n+')
    def NEWLINE(self, t):
        self.lineno += t.value.count('\n')
        return t

    def error(self, t):
        print(f"Carácter ilegal '{t.value[0]}' en la línea {t.lineno}")
        self.index += 1

# Ejemplo de uso
lexer = Lexer()
code = '''
fun quicksort(l:int, r:int, a:int[8192])
i:int;
j:int;
x:int;
w:int;
tmp:int;
done:int;
begin
i := l;
j := r;
x := a[(l+r)/2];
done := 0;
while done == 0 do
begin
while a[i] < x do
i := i + 1;
while x < a[j] do
j := j - 1;
if i <= j then
begin
tmp := a[i];
a[i] := a[j];
a[j] := tmp;

i := i + 1;
j := j - 1
end;
if i > j then
done := 1
end;
if l < j then
tmp := quicksort(l, j, a);
if i < r then
tmp := quicksort(i, r, a)
end
fun main()
v:int[8192];
i:int;
n:int;
begin
print("Entre n: ");
read(n);
i := 0;
while i < n do
begin
read(v[i]);
i := i+1
end;
quicksort(0, n-1, v);
i := 0;
while i < n-1 do
begin
write(v[i]); print(" ");
if 0 < v[i] - v[i+1] then
begin
print("Quicksort falló "); write(i); print("\n") ; return(0)
end
else
i := i+1
end;
write(v[i]);
print("Éxito\n")
end

'''
for token in lexer.tokenize(code):
    print(token)




Token(type='FUN', value='fun', lineno=2, index=1, end=4)
Token(type='QUICKSORT', value='quicksort', lineno=2, index=5, end=14)
Token(type='(', value='(', lineno=2, index=14, end=15)
Token(type='L', value='l', lineno=2, index=15, end=16)
Token(type=':', value=':', lineno=2, index=16, end=17)
Token(type='INT', value='int', lineno=2, index=17, end=20)
Token(type=',', value=',', lineno=2, index=20, end=21)
Token(type='R', value='r', lineno=2, index=22, end=23)
Token(type=':', value=':', lineno=2, index=23, end=24)
Token(type='INT', value='int', lineno=2, index=24, end=27)
Token(type=',', value=',', lineno=2, index=27, end=28)
Token(type='A', value='a', lineno=2, index=29, end=30)
Token(type=':', value=':', lineno=2, index=30, end=31)
Token(type='INT', value='int', lineno=2, index=31, end=34)
Token(type='[', value='[', lineno=2, index=34, end=35)
Token(type='INUMBER', value=8192, lineno=2, index=35, end=39)
Token(type=']', value=']', lineno=2, index=39, end=40)
Token(type=')', value=')', li

Estructura del AST

In [ ]:
from dataclasses import dataclass
from multimethod import multimeta


class Visitor(metaclass=multimeta):
  ...

@dataclass
class Node:
  def accept(self, v:Visitor, *args, **kwargs):
    return v.visit(self, *args, **kwargs)

@dataclass
class Expression(Node):
  ...

@dataclass
class Program(Node):
  instrucciones : list

@dataclass
class Unary(Expression):
  op : str
  expr : Expression

@dataclass
class Assign(Expression):
  IDENT : str
  expr : Expression

@dataclass
class Binary(Expression):
  op    : str
  left  : Expression
  right : Expression

@dataclass
class INumber(Expression):
  value : int

@dataclass
class FNumber(Expression):
  value : float

@dataclass
class While(Node):
    condition: Node
    stmtlist: list

@dataclass
class For(Node):
    ID: str
    start_expr: Node
    end_expr: Node
    stmtlist: list


@dataclass
class Literal(Expression):
	...
@dataclass
class StmtList(Node):
    stmtlist: list

@dataclass
class RangeExpr(Node):
    expression0: Expression
    expression1: Expression

@dataclass
class DoStmt(Node):
    stmtlist: list

@dataclass
class Param(Node):
    IDENT: str
    TYPE: str

@dataclass
class ParamList(Node):
    params: list

@dataclass
class ReturnStmt(Node):
    expr: Node


In [ ]:
class Eval:

  @classmethod
  def eval(cls, n):
      v = cls()
      return n.accept(v)

  def visit(self, n: Program):
      lista = []
      for instruccion in n.instrucciones:
          lista.append(instruccion.accept(self))
      return lista

  def visit(self, n: INumber):
      return n.value

  def visit(self, n: FNumber):
      return n.value

  def visit(self, n: Binary):
      left = n.left.accept(self)
      right = n.right.accept(self)
      if n.op == '^':
          return eval(f"{left} ** {right}")
      else:
          return eval(f"{left} {n.op} {right}")

  def visit(self, n: Assign):
      valor = n.expr.accept(self)
      return valor

  def visit(self, n: While):
      while n.condition.accept(self):
          for stmt in n.stmtlist:
              stmt.accept(self)

  def visit(self, n: For):
      start = n.start_expr.accept(self)
      end = n.end_expr.accept(self)
      ID = n.ID
      for i in range(start, end+1):
          self.agregar(ID, i)
          for stmt in n.stmtlist:
              stmt.accept(self)

  def visit(self, n: Assign):
      valor = n.expr.accept(self)
      self.agregar(n.IDENT, valor)
      return valor

  def visit(self, n: INumber):
      return n.value

  def visit(self, n: FNumber):
      return n.value

  def visit(self, n: Binary):
      left = n.left.accept(self)
      right = n.right.accept(self)
      if n.op == '^':
          return eval(f"{left} ** {right}")
      else:
          return eval(f"{left} {n.op} {right}")

  def visit(self, n: While):
      while n.condition.accept(self):
          for stmt in n.stmtlist:
              stmt.accept(self)

  def visit(self, n: For):
      start = n.start_expr.accept(self)
      end = n.end_expr.accept(self)
      ID = n.ID
      for i in range(start, end+1):
          self.agregar(ID, i)
          for stmt in n.stmtlist:
              stmt.accept(self)
  """
  def visit(self, n: IF):
        if n.condition.accept(self):
            for stmt in n.stmtlist_true:
                stmt.accept(self)
        else:
            for stmt in n.stmtlist_false:
                stmt.accept(self)
  """
  def visit(self, n: StmtList):
      for stmt in n.stmtlist:
          stmt.accept(self)

  def visit(self, n: RangeExpr):
      start = n.expression0.accept(self)
      end = n.expression1.accept(self)
      return range(start, end+1)

  def visit(self, n: DoStmt):
      for stmt in n.stmtlist:
          stmt.accept(self)

  def visit(self, n: ParamList):
      params = []
      for param in n.params:
          params.append(param.accept(self))
      return params

  def visit(self, n: Param):
      return (n.IDENT, n.TYPE)

  def visit(self, n: ReturnStmt):
        return n.expression.accept(self)


In [ ]:
from graphviz import Digraph

class Dot(Visitor):
  node_default = {
      'shape' : 'box',
      'color' : 'cyan',
      'style' : 'filled'
  }
  edge_default = {
      'arrowhead' : 'none',
  }

  def __init__(self):
    self.dot = Digraph('AST')
    self.dot.attr('node', **self.node_default)
    self.dot.attr('edge', **self.edge_default)
    self.seq = 0

  def __str__(self):
    return self.dot.source

  def __repr__(self):
    return self.dot.source

  def name(self):
    self.seq += 1
    return f'n{self.seq:0d}'

  @classmethod
  def render(cls, n:Node):
    dot = cls()
    n.accept(dot)
    return dot.dot
  def visit(self, n:Program):
    name = self.name()
    self.dot.node(name, label=f'Programa')
    for instruccion in n.instrucciones:
      self.dot.edge(name,instruccion.accept(self))


  def visit(self, n:INumber):
    name = self.name()
    self.dot.node(name, label=f'{n.value}')
    return name

  def visit(self, n:FNumber):
    name = self.name()
    self.dot.node(name, label=f'{n.value}')
    return name

  def visit(self, n:Unary):
    name = self.name()
    self.dot.node(name, label=f'{n.op}', shape='circle', color='darkseagreen1')
    self.dot.edge(name, n.expr.accept(self))
    return name

  def visit(self, n:Binary):
    name = self.name()
    self.dot.node(name, label=f'{n.op}', shape='circle', color='darkseagreen1')
    self.dot.edge(name, n.left.accept(self))
    self.dot.edge(name, n.right.accept(self))
    return name

  def visit(self, n:Assign):
    name = self.name()
    self.dot.node(name, label=f'=', shape='circle', color='darkseagreen1')
    self.dot.edge(name, n.IDENT)
    self.dot.edge(name, n.expr.accept(self))
    return name


  def visit(self, n: While):
        name = self.name()
        self.dot.node(name, label='While', shape='box', color='cyan', style='filled')
        self.dot.edge(name, n.condition.accept(self))
        for stmt in n.stmtlist:
            self.dot.edge(name, stmt.accept(self))
        return name

  def visit(self, n: For):
      name = self.name()
      self.dot.node(name, label='For', shape='box', color='cyan', style='filled')
      self.dot.edge(name, n.start_expr.accept(self))
      self.dot.edge(name, n.end_expr.accept(self))
      self.dot.edge(name, n.ID)
      for stmt in n.stmtlist:
          self.dot.edge(name, stmt.accept(self))
      return name
  """
  def visit(self, n: If):
      name = self.name()
      self.dot.node(name, label='If', shape='box', color='cyan', style='filled')
      self.dot.edge(name, n.condition.accept(self))
      for stmt in n.stmtlist_true:
          self.dot.edge(name, stmt.accept(self))
      for stmt in n.stmtlist_false:
          self.dot.edge(name, stmt.accept(self))
      return name
  """
  def visit(self, n: StmtList):
      name = self.name()
      self.dot.node(name, label='StmtList', shape='box', color='cyan', style='filled')
      for stmt in n.stmtlist:
          self.dot.edge(name, stmt.accept(self))
      return name

  def visit(self, n: RangeExpr):
      name = self.name()
      self.dot.node(name, label='RangeExpr', shape='box', color='cyan', style='filled')
      self.dot.edge(name, n.expression0.accept(self))
      self.dot.edge(name, n.expression1.accept(self))
      return name

  def visit(self, n: DoStmt):
      name = self.name()
      self.dot.node(name, label='DoStmt', shape='box', color='cyan', style='filled')
      for stmt in n.stmtlist:
          self.dot.edge(name, stmt.accept(self))
      return name

  def visit(self, n: ParamList):
      name = self.name()
      self.dot.node(name, label='ParamList', shape='box', color='cyan', style='filled')
      for param in n.params:
          self.dot.edge(name, param.accept(self))
      return name

  def visit(self, n: Param):
      name = self.name()
      self.dot.node(name, label=f'Param\n{str(n)}', shape='box', color='cyan', style='filled')
      return name

  def visit(self, n: ReturnStmt):
      name = self.name()
      self.dot.node(name, label='ReturnStmt', shape='box', color='cyan', style='filled')
      self.dot.edge(name, n.expr.accept(self))
      return name

# Analizador sintactico

In [ ]:
class Parser(sly.Parser):
    debugfile = 'eval.txt'  # Correr el programa
    tokens = Lexer.tokens

    def __init__(self):
        self.mem_dict = {}

    def agregar(self, IDENT, expr):
        self.mem_dict[IDENT] = expr

    def sacar_valor(self, IDENT):
        return self.mem_dict.get(IDENT)

    @_('stmtlist')
    def programa(self, p):
        return Program(p.stmtlist)

    @_('stmtlist ";" stmt')
    def stmtlist(self, p):
        p.stmtlist.append(p.stmt)
        return p.stmtlist

    @_('stmt')
    def stmtlist(self, p):
        return [p.stmt]

    @_('FUN IDENT "(" param_list ")" BEGIN stmtlist END')
    def FUN(self, p):
        return Funcion(p.IDENT, p.param_list, p.stmtlist)

    @_('MAIN "(" ")" BEGIN stmtlist END')
    def MAIN(self, p):
        return Funcion("main", [], p.stmtlist)

    @_('param_list "," param')
    def param_list(self, p):
        return p.param_list + [p.param]

    @_('param')
    def param_list(self, p):
        return [p.param]

    @_('IDENT ":" TYPE ";"')
    def param(self, p):
        return (p.IDENT, p.TYPE)

    @_("ASSIGN")
    def ASSIGN(self, IDENT, expr):
      self.agregar(IDENT, expr)
      return ASSIGN(IDENT, expr)

    @_('IDENT')
    def param(self, p):
        return (p.IDENT,)

    @_('while_stmt')
    def stmt(self, p):
        return p.while_stmt

    @_('for_stmt')
    def stmt(self, p):
        return p.for_stmt

    @_('if_stmt')
    def stmt(self, p):
        return p.if_stmt

    @_('begin_stmt')
    def stmt(self, p):
        return p.begin_stmt

    @_('BEGIN stmtlist END')
    def begin_stmt(self, p):
        return p.stmtlist

    @_('IF "(" expression ")" THEN stmtlist ELSE stmtlist END')
    def if_stmt(self, p):
        return If(p.expression, p.stmtlist0, p.stmtlist1)

    @_('WHILE "(" expression ")" DO stmtlist END')
    def while_stmt(self, p):
        return While(p.expression, p.stmtlist)

    @_('FOR IDENT IN range_expr DO stmtlist END')
    def for_stmt(self, p):
        return For(p.IDENT, p.range_expr[0], p.range_expr[1], p.stmtlist)

    @_("FOR IDENT IN range_expr DOSPUNTOS stmtlist END")
    def For(self, p):
        return For(p.IDENT, p.range_expr[0], p.range_expr[1], p.stmtlist)

    @_("WHILE '(' expression ')' DOSPUNTOS stmtlist END")
    def While(self, p):
        return While(p.expression, p.stmtlist)

    @_('"(" expression "," expression ")"')
    def range_expr(self, p):
        return (p.expression0, p.expression1)

    @_("term '+' expr",
       "term '-' expr")
    def expr(self, p):
        return Binary(p[1], p.term, p.expr)

    @_("term")
    def expr(self, p):
        return p.term

    @_("factor '*' term",
       "factor '/' term")
    def term(self, p):
        return Binary(p[1], p.factor, p.term)

    @_('STRING')
    def STRING(self, p):
        string_value = p.STRING[1:-1]
        return STRING(string_value)

    @_("expr GT expr",
      "expr LT expr",
      "expr EQ expr",
      "expr NE expr",
      "expr GE expr",
      "expr LE expr")
    def expression(self, p):
        return Binary(p[2], p.expr0, p.expr1)

    @_("factor")
    def term(self, p):
        return p.factor

    @_("INUMBER")
    def factor(self, p):
        return INumber(p.INUMBER)

    @_("FNUMBER")
    def factor(self, p):
        return FNumber(p.FNUMBER)

    @_('DOSPUNTOS')
    def DOSPUNTOS(self, p):
        pass

    @_("expr")
    def stmt(self, p):
        return p.expr

    @_("IDENT '=' expr")
    def stmt(self, p):
        self.agregar(p.IDENT, p.expr)
        return Assign(p.IDENT, p.expr)

    @_("IDENT")
    def factor(self, p):
        return self.sacar_valor(p.IDENT)

    @_("'(' expr ')'")
    def factor(self, p):
        return p.expr

    @_('BREAK')
    def BREAK(self, p):
        return BREAK()

    @_('RETURN expr')
    def return_stmt(self, p):
        return Return(p.expr)

    @_('PRINT "(" expr ")"')
    def PRINT(self, p):
        return PRINT(p.expr)

    @_('LOCATION "(" STRING ")"')
    def LOCATION(self, p):
        return LOCATION(p.STRING)

    @_('READ "(" IDENT ")"')
    def read(self, p):
        variable_name = p.IDENT
        return Read(variable_name)

    @_('WRITE "(" expr ")"')
    def write(self, p):
        expression = p.expr
        return Write(expression)

    @_('SKIP')
    def skip(self, p):
        return Skip()

    @_('NEWLINE')
    def newline(self, p):
        pass


YaccError: ignored

In [ ]:
code = 'a = 1 + 2 * 3 / 4'
lex = Lexer()
pas = Parser()
a=(lex.tokenize(code))
for tok in a:
  print(tok)
ast = pas.parse(lex.tokenize(code))
print(ast)
Dot.render(ast)

NameError: ignored

# Analizador Semantico

In [ ]:
class Symtab:
  '''
  Una tabla de símbolos.  Este es un objeto simple que sólo
  mantiene una hashtable (dict) de nombres de simbolos y los
  nodos de declaracion o definición de funciones a los que se
  refieren.
  Hay una tabla de simbolos separada para cada elemento de
  código que tiene su propio contexto (por ejemplo cada función,
  clase, tendra su propia tabla de simbolos). Como resultado,
  las tablas de simbolos se pueden anidar si los elementos de
  código estan anidados y las búsquedas de las tablas de
  simbolos se repetirán hacia arriba a través de los padres
  para representar las reglas de alcance léxico.
  '''
  class SymbolDefinedError(Exception):
    '''
    Se genera una excepción cuando el código intenta agregar
    un simbol a una tabla donde el simbol ya se ha definido.
    Tenga en cuenta que 'definido' se usa aquí en el sentido
    del lenguaje C, es decir, 'se ha asignado espacio para el
    simbol', en lugar de una declaración.
    '''
    pass

  def __init__(self, parent=None):
    '''
    Crea una tabla de símbolos vacia con la tabla de
    simbolos padre dada.
    '''
    self.entries = {}
    self.parent = parent
    if self.parent:
      self.parent.children.append(self)
    self.children = []

  def add(self, name, value):
    '''
    Agrega un simbol con el valor dado a la tabla de simbolos.
    El valor suele ser un nodo AST que representa la declaración
    o definición de una función, variable (por ejemplo, Declaración
    o FuncDeclaration)
    '''
    if name in self.entries:
      raise Symtab.SymbolDefinedError()
    self.entries[name] = value

  def get(self, name):
    '''
    Recupera el símbol con el nombre dado de la tabla de
    simbol, recorriendo hacia arriba a traves de las tablas
    de simbol principales si no se encuentra en la actual.
    '''
    if name in self.entries:
      return self.entries[name]
    elif self.parent:
      return self.parent.get(name)
    return None

In [ ]:
class Checker(Visitor):

    def visit(self, n: Literal, env: Symtab):
      # Devolver datatype
        return n.datatype

    def visit(self, n: Location, env: Symtab):
      # Buscar en Symtab y extraer datatype (No se encuentra?)
		# Devuelvo el datatype
        datatype = env.get(n.IDENT)
        if not datatype:
            raise NameError(f"La variable {n.IDENT} no está definida")
        return datatype

    def visit(self, n: TypeCast, env: Symtab):
      # Visitar la expresion asociada
		# Devolver datatype asociado al nodo
        expr_datatype = n.expr.accept(self, env)
        if n.TYPE != expr_datatype:
            raise TypeError(f"Error de tipo en la conversión: {n.TYPE} y {expr_datatype}")
        return n.TYPE

    def visit(self, n: Assign, env: Symtab):
      # Visitar el hijo izquierdo (devuelve datatype)
		# Visitar el hijo derecho (devuelve datatype)
		# Comparar ambos tipo de datatype
        left_datatype = n.left.accept(self, env)
        right_datatype = n.right.accept(self, env)

        if left_datatype != right_datatype:
            raise TypeError(f"Tipos incompatibles en la asignación: {left_datatype} y {right_datatype}")

        return left_datatype

    def visit(self, n: FuncCall, env: Symtab):
      # Buscar la funcion en Symtab (extraer: Tipo de retorno, el # de parametros)
		# Visitar la lista de Argumentos
		# Comparar el numero de argumentos con parametros
		# Comparar cada uno de los tipos de los argumentos con los parametros
		# Retornar el datatype de la funcion
        func = env.get(n.IDENT)
        if not func or not isinstance(func, FuncDefinition):
            raise NameError(f"La función {n.IDENT} no está definida")

        if len(n.arglist) != len(func.paramlist):
            raise TypeError(f"Número incorrecto de argumentos para la función {n.IDENT}")

        for arg, param in zip(n.arglist, func.paramlist):
            arg_type = arg.accept(self, env)
            if arg_type != param.TYPE:
                raise TypeError(f"Tipo incorrecto para el argumento en la llamada a la función {n.IDENT}")

        return func.return_type

    def visit(self, n: Binary, env: Symtab):
      # Visitar el hijo izquierdo (devuelve datatype)
		# Visitar el hijo derecho (devuelve datatype)
		# Comparar ambos tipo de datatype
        left_datatype = n.left.accept(self, env)
        right_datatype = n.right.accept(self, env)

        if n.op in {'+', '-', '*', '/'}:
            if left_datatype not in {'int', 'float'} or right_datatype not in {'int', 'float'}:
                raise TypeError(f"Operación no válida para tipos {left_datatype} y {right_datatype}")
            if left_datatype == 'int' and right_datatype == 'int':
                return 'int'
            else:
                return 'float'
        elif n.op == '^':
            if left_datatype != 'int' or right_datatype != 'int':
                raise TypeError(f"Operación no válida para tipos {left_datatype} y {right_datatype}")
            return 'int'
        else:
            raise ValueError(f"Operador no reconocido: {n.op}")

    def visit(self, n: Logical, env: Symtab):
      # Visitar el hijo izquierdo (devuelve datatype)
		# Visitar el hijo derecho (devuelve datatype)
		# Comparar ambos tipo de datatype
        left_datatype = n.left.accept(self, env)
        right_datatype = n.right.accept(self, env)

        if n.op in {'and', 'or'}:
            if left_datatype != 'bool' or right_datatype != 'bool':
                raise TypeError(f"Operación no válida para tipos {left_datatype} y {right_datatype}")
            return 'bool'
        else:
            raise ValueError(f"Operador lógico no reconocido: {n.op}")

    def visit(self, n: Unary, env: Symtab):
      # Visitar la expression asociada (devuelve datatype)
		# Comparar datatype
        expr_datatype = n.expr.accept(self, env)
        if n.op == '!':
            if expr_datatype != 'int':
                raise TypeError(f"Operación no válida para tipo {expr_datatype}")
            return 'int'
        else:
            raise ValueError(f"Operador unario no reconocido: {n.op}")

    def visit(self, n: FuncDefinition, env: Symtab):
        # Agregar el nombre de la funcion a Symtab
		# Crear un nuevo contexto (Symtab)
		# Visitar ParamList, VarList, StmtList
		# Determinar el datatype de la funcion (revisando instrucciones return)
        env.add(n.IDENT, n)
        # Crea un nuevo contexto (Symtab)
        local_env = Symtab(parent=env)
        # Visitar ParamList, VarList, StmtList
        for param in n.paramlist:
            param.accept(self, local_env)
        for var in n.varlist:
            var.accept(self, local_env)
        n.stmtlist.accept(self, local_env)
        # Determinar el datatype de la función (revisando instrucciones return)
        return_type = None
        for stmt in n.stmtlist:
            if isinstance(stmt, Return):
                return_type = stmt.accept(self, local_env)
        n.return_type = return_type

    def visit(self, n: VarDefinition, env: Symtab):
        # Agrega la lógica para manejar la definición de variables
        env.add(n.IDENT, n.TYPE)

    def visit(self, n: Parameter, env: Symtab):
       # Agregar el nombre del parametro a Symtab
        env.add(n.IDENT, n.TYPE)

    def visit(self, n: Print, env: Symtab):
        expr_datatype = n.expr.accept(self, env)
        if expr_datatype not in {'int', 'float', 'string'}:
            raise TypeError(f"Tipo no válido para la instrucción print: {expr_datatype}")

    def visit(self, n: Write, env: Symtab):
        # Buscar la Variable en Symtab
        expr_datatype = n.location.accept(self, env)
        if expr_datatype not in {'int', 'float'}:
            raise TypeError(f"Tipo no válido para la instrucción write: {expr_datatype}")

    def visit(self, n: Read, env: Symtab):
        # Buscar la Variable en Symtab
        location_datatype = n.location.accept(self, env)
        if location_datatype not in {'int', 'float'}:
            raise TypeError(f"Tipo no válido para la instrucción read: {location_datatype}")

    def visit(self, n: While, env: Symtab):
        # Visitar la condicion del While (Comprobar tipo bool)
		  # Visitar las Stmts
        condition_datatype = n.condition.accept(self, env)

        # Verificar que la condición sea de tipo bool
        if condition_datatype != 'bool':
            raise TypeError(f"La condición del bucle 'while' debe ser de tipo 'bool', no {condition_datatype}")

        # Crear un nuevo entorno para las instrucciones dentro del bucle
        while_env = Symtab(parent=env)

        # Visitar las instrucciones dentro del bucle con el nuevo entorno
        n.stmtlist.accept(self, while_env)

    def visit(self, n: Break, env: Symtab):
        # Esta dentro de un While?
        current_env = env
        while current_env:
            if isinstance(current_env.entries.get('loop'), While):
                return
            current_env = current_env.parent
        raise ValueError("La instrucción 'break' debe estar dentro de un bucle 'while'")

    def visit(self, n: IfStmt, env: Symtab):
        # Visitar la condicion del IfStmt (Comprobar tipo bool)
		    # Visitar las Stmts del then y else
        condition_datatype = n.condition.accept(self, env)
        if condition_datatype != 'bool':
            raise TypeError(f"Tipo no válido para la condición del IfStmt: {condition_datatype}")

        # Stmts del 'then' y 'else'
        n.stmtlist_then.accept(self, env)
        if n.stmtlist_else:
            n.stmtlist_else.accept(self, env)

    def visit(self, n: Return, env: Symtab):
        # Visitar la expresion asociada
		# Actualizar el datatype de la funcion
        expr_datatype = n.expr.accept(self, env)

        func_def = env.get('current_function')
        if func_def:
            if not func_def.return_type:
                func_def.return_type = expr_datatype
            else:
                if func_def.return_type != expr_datatype:
                    raise TypeError(f"Tipo de retorno incompatible para la función {func_def.IDENT}")

    def visit(self, n: Skip, env: Symtab):
        pass

    def visit(self, n: Program, env: Symtab):
        # Crear un nuevo contexto (Symtab global)
		# Visitar cada una de las declaraciones asociadas
        global_env = Symtab(parent=None)
        n.symtab = global_env
        #visitar cada una de las declaraciones asociadas
        for declaration in n.declarations:
            declaration.accept(self, global_env)

    def visit(self, n: StmtList, env: Symtab):
        # Visitar cada una de las instruciones asociadas
        for stmt in n.stmts:
            stmt.accept(self, env)

    def visit(self, n: VarList, env: Symtab):
        # Visitar cada una de las variables asociadas
        for var in n.vars:
            var.accept(self, env)

    def visit(self, n: ParmList, env: Symtab):
        # Visitar cada una de los parametros asociados
        for param in n.params:
            param.accept(self, env)

    def visit(self, n: ArgList, env: Symtab):
        # Visitar cada una de los argumentos asociados
        for arg in n.args:
            arg.accept(self, env)

NameError: ignored

In [ ]:
from rich import print

In [ ]:
code = 'a = 1 + 2 * 3 / 4'
lex = Lexer()
pas = Parser()
a=(lex.tokenize(code))
for tok in a:
  print(tok)
ast = pas.parse(lex.tokenize(code))
print(ast)
Dot.render(ast)

NameError: ignored

In [ ]:
code = 'a = 1 + 2 * 3 / 4; a / 2; b = sin(3*pi/8) + a'

lex = Lexer()
pas = Parser()
a=(lex.tokenize(code))
for tok in a:
  print(tok)
ast = pas.parse(lex.tokenize(code))
print(ast)
check = Checker()
symtab = Symtab()
check.visit(ast, symtab)
Dot.render(ast)

NameError: ignored

In [ ]:
def main(argv):
    if len(argv) != 2:
        print(f"Usage: python {argv[0]} filename")
        exit(1)

    lex = Lexer()
    txt = open(argv[1]).read()

    for tok in lex.tokenize(txt):
        print(tok)


if __name__ == '__main__':
    from sys import argv
    main(argv)